In [6]:
import nltk
nltk.download("punkt")
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.optimizers import SGD
import random

words = []
classes = []
documents = []
ignore_words = ["?","!","@","$"]

data_file = open("intents.json").read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")



15 documents
5 classes ['About', 'Exe-com', 'Name', 'Why IEEE', 'greetings']
53 unique lemmatized words ['a', 'about', 'academic', 'activity', 'all', 'an', 'and', 'any', 'are', 'available', 'become', 'club', 'com', 'do', 'doe', 'exactly', 'exe', 'exe-com', 'for', 'form', 'full', 'good', 'hello', 'help', 'hi', 'how', 'ieee', 'in', 'is', 'it', 'join', 'kind', 'leadership', 'member', 'morning', 'not', 'of', 'offer', 'opportunity', 'other', 'our', 'second', 'stand', 'student', 'the', 'there', 'to', 'what', 'why', 'will', 'within', 'year', 'you']
Training data created


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
<ipython-input-6-d4881b832c8f>:69: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [7]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
3/3 [==============================] - 1s 7ms/step - loss: 1.6282 - accuracy: 0.3333
Epoch 2/200
3/3 [==============================] - 0s 5ms/step - loss: 1.7855 - accuracy: 0.0000e+00
Epoch 3/200
3/3 [==============================] - 0s 4ms/step - loss: 1.5449 - accuracy: 0.2667
Epoch 4/200
3/3 [==============================] - 0s 4ms/step - loss: 1.4810 - accuracy: 0.3333
Epoch 5/200
3/3 [==============================] - 0s 4ms/step - loss: 1.4800 - accuracy: 0.4667
Epoch 6/200
3/3 [==============================] - 0s 4ms/step - loss: 1.3962 - accuracy: 0.4667
Epoch 7/200
3/3 [==============================] - 0s 4ms/step - loss: 1.3408 - accuracy: 0.4667
Epoch 8/200
3/3 [==============================] - 0s 4ms/step - loss: 1.3110 - accuracy: 0.4000
Epoch 9/200
3/3 [==============================] - 0s 7ms/step - loss: 1.1037 - accuracy: 0.6000
Epoch 10/200
3/3 [==============================] - 0s 4ms/step - loss: 1.0554 - accuracy: 0.5333
Epoch 11/200
3/3 [=======